In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import math
import random
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv('../input/spaceship-titanic/train.csv')
test = pd.read_csv('../input/spaceship-titanic/test.csv')

In [ ]:
def randString(n):
  import secrets
  import string
  res = ''.join(secrets.choice(string.ascii_uppercase + string.digits) for i in range(n))
  return res    

In [ ]:
train['Name'] = train['Name'].apply(lambda x: f'{randString(5)} {randString(5)}' 
                                          if type(x) == float else x)
test['Name'] = test['Name'].apply(lambda x: f'{randString(5)} {randString(5)}' 
                                          if type(x) == float else x)

In [ ]:
temp = train.append(test)
temp = temp.reset_index()
temp['dv_lastName'] = ''
for i in range(len(temp)):
  name = temp['Name'][i].split(' ')
  temp['dv_lastName'][i] = name[1]
temp = pd.get_dummies(temp, columns=['dv_lastName'])
temp.drop('index',inplace=True,axis=1)

In [ ]:
train_df = temp[0:8693]
test_df = temp[8693:]
train_df.index = train.index
test_df.index = test.index

In [ ]:
null_numerical_vars = ['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
mean_var_dict = {}
for col in null_numerical_vars:
    mean_var_dict[col] = (np.mean(train_df[col]), np.std(train_df[col]))
for col in null_numerical_vars:
    train_df[col].fillna('NA',inplace=True)
    test_df[col].fillna('NA',inplace=True)
    
    mu, sigma = mean_var_dict[col]
    train_df[col] = train_df[col].apply(lambda x: np.random.normal(mu,sigma,1)
                                          if type(x) == str else x)
    test_df[col] = test_df[col].apply(lambda x: np.random.normal(mu,sigma,1) 
                                          if type(x) == str else x)

In [ ]:
null_categorical_vars = ['HomePlanet','Cabin','Destination']
for col in null_categorical_vars:
    train_df[col].fillna(train_df[col].mode().iloc[0],inplace=True)
    test_df[col].fillna(test_df[col].mode().iloc[0],inplace=True)

In [ ]:
train_df['CryoSleep'].fillna(train_df['CryoSleep'].value_counts().index[0],inplace=True)
test_df['CryoSleep'].fillna(test_df['CryoSleep'].value_counts().index[0],inplace=True)

train_df['VIP'].fillna(train_df['VIP'].value_counts().index[0],inplace=True)
test_df['VIP'].fillna(test_df['VIP'].value_counts().index[0],inplace=True)

In [ ]:
train_df['dv_Group'] = ''
for i in range(len(train_df)):
  train_df['dv_Group'][i] = int(train_df['PassengerId'][i][-2:])

test_df['dv_Group'] = ''
for i in range(len(test_df)):
  test_df['dv_Group'][i] = int(test_df['PassengerId'][i][-2:])

In [ ]:
train_df['dv_deck'] = ''
train_df['dv_num'] = ''
train_df['dv_side'] = ''
for i in range(len(train_df)):
  cab_split = train_df['Cabin'][i].split('/')
  train_df['dv_deck'][i] = cab_split[0]
  train_df['dv_num'][i] = int(cab_split[1])
  train_df['dv_side'][i] = cab_split[2]
    
test_df['dv_deck'] = ''
test_df['dv_num'] = ''
test_df['dv_side'] = ''
for i in range(len(test_df)):
  cab_split = test_df['Cabin'][i].split('/')
  test_df['dv_deck'][i] = cab_split[0]
  test_df['dv_num'][i] = int(cab_split[1])
  test_df['dv_side'][i] = cab_split[2]

In [ ]:
cols_to_onehot = ['HomePlanet','Destination','dv_deck','dv_side']
cols_to_TF = ['CryoSleep','VIP','Transported']

In [ ]:
for col in cols_to_onehot:
    train_df[pd.get_dummies(train_df[col], prefix=col).columns] = pd.get_dummies(train_df[col], prefix=col)
for col in cols_to_onehot:
    test_df[pd.get_dummies(test_df[col], prefix=col).columns] = pd.get_dummies(test_df[col], prefix=col)

In [ ]:
cols_to_drop = ['Cabin','Name','PassengerId','HomePlanet','Destination','dv_deck','dv_side']
for col in cols_to_drop:
    train_df.drop(col,inplace=True,axis=1)
    test_df.drop(col,inplace=True,axis=1)

In [ ]:
for col in cols_to_TF:
    train_df[col] = train_df[col].astype(int)
for col in ['VIP','CryoSleep']:
    test_df[col] = test_df[col].astype(int)

In [ ]:
train_df.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
x = train_df.drop('Transported',axis=1)
y = train_df['Transported']

In [ ]:
parameters = {
    "n_estimators": [5, 10, 15, 20, 25], 
    "max_depth": [3, 5, 7, 9, 11, 12],
    "min_samples_leaf" : [18, 19, 20],
    "min_samples_split" : [8, 9, 10]
}


RFC_Model = RandomForestClassifier()
RFC_grid = GridSearchCV(RFC_Model, param_grid = parameters, cv = 5, scoring = 'accuracy', n_jobs= -1)
RFC_grid.fit(x, y)
print('Best Parameters : ', RFC_grid.best_params_)
print()
print('Best Accuracy : ', RFC_grid.best_score_)

In [ ]:
rf = RandomForestClassifier(max_depth= 12, min_samples_leaf= 19, min_samples_split= 8, n_estimators= 25)
rf.fit(x,y)
predictions = rf.predict(test_df)

In [ ]:
temp = []
for x in predictions:
    if x:
        temp.append(True)
    else:
        temp.append(False)

In [ ]:
subs = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')
subs['Transported'] = temp
subs.to_csv('submission.csv', index = False)